# <center>Contrôle Continue MDI721</center>

### Exercice:
Construire une base de données propre de conditionnement pour chaque vendeurs depuis open medicaments et à chaque fois extraire son dosage, sa forme galénique, sa posologie... afin de comparer les traitements.

In [1]:
# coding: utf-8
# Imports:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import json
import re

In [2]:
url = "https://www.open-medicaments.fr/api/v1/medicaments?query=paracetamol&limit=100"
#url = "https://www.open-medicaments.fr/api/v1/medicaments?query=paracetamol%20500"

In [3]:
page = get(url)
df = pd.read_json(page.content)

In [4]:
df.head()

,codeCIS,denomination
0,67445776,"PARACETAMOL ZYDUS 500 mg, gélule"
1,62721886,"PARACETAMOL RATIOPHARM 1000 mg, comprimé"
2,62772966,"PARACETAMOL EG 1 g, comprimé"
3,67034675,"PARACETAMOL SANDOZ 500 mg, gélule"
4,61223352,"PARACETAMOL RATIOPHARM 500 mg, gélule"


In [5]:
# Pour extraire la forme galénique:
regex_forme = r",([\w\s]*)"
denomination = "PARACETAMOL ZYDUS 500 mg, gélule"
forme = re.findall(regex_forme, denomination)
print(f"Forme galénique : {forme}")

Forme galénique : [' gélule']


In [6]:
# Pour extraire toutes les informations :
regex = r"([\D]*)(\d+)(.*),(.*)"
denomination = "PARACETAMOL ZYDUS 500 mg, gélule"
res = re.findall(regex, denomination)
print(f"Résultat : {res}")

Résultat : [('PARACETAMOL ZYDUS ', '500', ' mg', ' gélule')]


In [7]:
# Application sur toute la série
serie = df["denomination"]
res = serie.str.extract(regex)
res.head()

,0,1,2,3
0,PARACETAMOL ZYDUS,500,mg,gélule
1,PARACETAMOL RATIOPHARM,1000,mg,comprimé
2,PARACETAMOL EG,1,g,comprimé
3,PARACETAMOL SANDOZ,500,mg,gélule
4,PARACETAMOL RATIOPHARM,500,mg,gélule


In [8]:
res["mul"] = 1000
res["mul"] = res["mul"].where(res[2].str.strip() == "g", 1)
res["dosage"] = res[1].fillna(0).astype(int) * res["mul"]
res.head()

,0,1,2,3,mul,dosage
0,PARACETAMOL ZYDUS,500,mg,gélule,1,500
1,PARACETAMOL RATIOPHARM,1000,mg,comprimé,1,1000
2,PARACETAMOL EG,1,g,comprimé,1000,1000
3,PARACETAMOL SANDOZ,500,mg,gélule,1,500
4,PARACETAMOL RATIOPHARM,500,mg,gélule,1,500


In [9]:
res = res.drop(columns=["mul"])
res.head()

,0,1,2,3,dosage
0,PARACETAMOL ZYDUS,500,mg,gélule,500
1,PARACETAMOL RATIOPHARM,1000,mg,comprimé,1000
2,PARACETAMOL EG,1,g,comprimé,1000
3,PARACETAMOL SANDOZ,500,mg,gélule,500
4,PARACETAMOL RATIOPHARM,500,mg,gélule,500


In [19]:
# Mise en forme de la dataframe:
res = res.drop(columns=[1, 2])
res["grammage"] = "mg"
res = res.rename(columns = {0: "Nom medicaments", 3: "Forme"})
res.head()

,Nom medicaments,Forme,dosage,grammage
0,PARACETAMOL ZYDUS,gélule,500,mg
1,PARACETAMOL RATIOPHARM,comprimé,1000,mg
2,PARACETAMOL EG,comprimé,1000,mg
3,PARACETAMOL SANDOZ,gélule,500,mg
4,PARACETAMOL RATIOPHARM,gélule,500,mg


In [20]:
def main():
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(res)

if __name__ == '__main__':
    main()

                                      Nom medicaments  \
0                                  PARACETAMOL ZYDUS    
1                             PARACETAMOL RATIOPHARM    
2                                     PARACETAMOL EG    
3                                 PARACETAMOL SANDOZ    
4                             PARACETAMOL RATIOPHARM    
5                                  PARACETAMOL ARROW    
6                                   PARACETAMOL AHCL    
7                                 PARACETAMOL ISOMED    
8                             PARACETAMOL RATIOPHARM    
9                               PARACETAMOL EVOLUGEN    
10                       PARACETAMOL ZENTIVA CONSEIL    
11                              PARACETAMOL CRISTERS    
12                               PARACETAMOL RANBAXY    
13                               PARACETAMOL RANBAXY    
14                               PARACETAMOL ZENTIVA    
15                                 PARACETAMOL ZYDUS    
16                             